In [2]:
train = pd.read_csv("../labeledTrainData.tsv", delimiter='\t', quoting=3)
train.tail()

,id,sentiment,review
24995,"""3453_3""",0,"""It seems like more consideration has gone int..."
24996,"""5064_1""",0,"""I don't believe they made this film. Complete..."
24997,"""10905_3""",0,"""Guy is a loser. Can't get girls, needs to bui..."
24998,"""10194_3""",0,"""This 30 minute documentary Buñuel made in the..."
24999,"""8478_8""",1,"""I saw this movie as a child and it broke my h..."


In [3]:
test = pd.read_csv("../testData.tsv", delimiter="\t", quoting=3)
test.tail()

,id,review
24995,"""2155_10""","""Sony Pictures Classics, I'm looking at you! S..."
24996,"""59_10""","""I always felt that Ms. Merkerson had never go..."
24997,"""2531_1""","""I was so disappointed in this movie. I am ver..."
24998,"""7772_8""","""From the opening sequence, filled with black ..."
24999,"""11465_10""","""This is a great horror film for people who do..."


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
id           25000 non-null object
sentiment    25000 non-null int64
review       25000 non-null object
dtypes: int64(1), object(2)
memory usage: 586.0+ KB


In [5]:
train.sentiment.value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

In [6]:
train['review'][0][:700]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely lik'

html태그가 혼재해있어 정제해줄 필요가 있다.

### Data Cleaning and Text Preprocessing
1. BeautifulSoup을 통해 HTML태그 제거
2. 정규표현식으로 알파벳 이외의 문자를 공백으로 치환
3. NLTk데이터를 사용해 불용어(Stopword) 제거 (eg: I my me...)
4. 어간추출(Stemming), 음소표기법(Lemmatizing)의 개념을 이해하고 SnowballStemmer를 통해 어간 추출

In [7]:
!pip show BeautifulSoup4

Name: beautifulsoup4
Version: 4.6.0
Summary: Screen-scraping library
Home-page: http://www.crummy.com/software/BeautifulSoup/bs4/
Author: Leonard Richardson
Author-email: leonardr@segfault.org
License: MIT
Location: c:\programdata\anaconda3\lib\site-packages
Requires: 
Required-by: conda-build


##### 1. BeautifulSoup을 통해 HTML태그 제거

In [8]:
from bs4 import BeautifulSoup

In [9]:
example1 = BeautifulSoup(train['review'][0], "html5lib")

print(train['review'][0][:700])
example1.get_text()[:700]

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely lik


'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyw'

##### 2. 정규표현식으로 특수문자 제거

In [10]:
import re

In [11]:
letters_only = re.sub('[^a-zA-Z]',' ', example1.get_text())
letters_only[:700]

' With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyw'

In [12]:
# 모두 소문자로 나눈다
lower_case = letters_only.lower()

In [13]:
# 문자를 나눈다 (=토큰화한다.)
words = lower_case.split()
print(len(words))
words[:10]

437


['with',
 'all',
 'this',
 'stuff',
 'going',
 'down',
 'at',
 'the',
 'moment',
 'with']

##### 3. 불용어(Stopword) 제거
- 실제로 많이 등장하는 단어지만 분석하는데 의미없는...(I,my,the,....)
- 예측하는데 실제로 기여하지 않아 다른 텍스트와 구별하지 못한다.

In [14]:
import nltk
from nltk.corpus import stopwords
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [15]:
# stopwords를 제거한 토큰들
words = [word for word in words if not word in stopwords.words('english')]
print(len(words))
words[:10]

219


['stuff',
 'going',
 'moment',
 'mj',
 'started',
 'listening',
 'music',
 'watching',
 'odd',
 'documentary']

439 -> 219개로 감소

##### 4-1. Stemming(어간 추출. 형태소 분석)
- cats, catlike, catty.... --> 어간 'cat'추출 

In [16]:
# 포터 스테머 사용
porter_stemmer = nltk.stem.PorterStemmer()
print(porter_stemmer.stem('maximum'))
print("The stemmed form of running: {}".format(porter_stemmer.stem("running")))
print("The stemmed form of runs: {}".format(porter_stemmer.stem("runs")))
print("The stemmed form of run: {}".format(porter_stemmer.stem("run")))

maximum
The stemmed form of running: run
The stemmed form of runs: run
The stemmed form of run: run


In [17]:
# 랭커스터 스테머 사용
from nltk.stem.lancaster import LancasterStemmer
lancaster_stemmer = LancasterStemmer()
print(lancaster_stemmer.stem('maximum'))
print("The stemmed form of running: {}".format(lancaster_stemmer.stem("running")))
print("The stemmed form of runs: {}".format(lancaster_stemmer.stem("runs")))
print("The stemmed form of run: {}".format(lancaster_stemmer.stem("run")))

maxim
The stemmed form of running: run
The stemmed form of runs: run
The stemmed form of run: run


In [18]:
# 스노우볼 스테머
from nltk.stem.snowball import SnowballStemmer
snowball_stemmer = SnowballStemmer('english')
print(snowball_stemmer.stem('maximum'))
print("The stemmed form of running: {}".format(snowball_stemmer.stem("running")))
print("The stemmed form of runs: {}".format(snowball_stemmer.stem("runs")))
print("The stemmed form of run: {}".format(snowball_stemmer.stem("run")))

maximum
The stemmed form of running: run
The stemmed form of runs: run
The stemmed form of run: run


SnowballStemmer를 사용하도록 하겠슴.

In [19]:
stemmer = SnowballStemmer('english')
words = [stemmer.stem(word) for word in words]
words[:10]

['stuff',
 'go',
 'moment',
 'mj',
 'start',
 'listen',
 'music',
 'watch',
 'odd',
 'documentari']

going -> go, starting -> start....바뀜

##### 4-2. Lemmatization(음소표기법)
- 동음이의어가 문맥에 따라 다른 의미를 갖는데, 이를 앞뒤 문맥을 보고 식별

In [20]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [21]:
print(wordnet_lemmatizer.lemmatize('fly'))
print(wordnet_lemmatizer.lemmatize('flies'))

fly
fly


In [22]:
words = [wordnet_lemmatizer.lemmatize(word) for word in words]
words[:10]

['stuff',
 'go',
 'moment',
 'mj',
 'start',
 'listen',
 'music',
 'watch',
 'odd',
 'documentari']

#### 문자열 처리과정을 함수화 해보자

In [23]:
def review_to_words(raw_review):
    # html 제거
    review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
    # 영문자가 아닌 문자는 공백으로 변환
    letter_only = re.sub('[^a-zA-Z]', " ", review_text)
    # 소문자 변환, 토큰화
    words = letters_only.lower().split()
    
    # stopword 제거
    # 처리속도 향상을 위해 stopword를 list -> set으로 변환
    stopword = set(stopwords.words("english"))
    meaningful_words = [word for word in words if not word in stopword]
    
    # 어간 추출
    stemming_words = [stemmer.stem(word) for word in meaningful_words]
    
    # 공백으로 구분된 문자열로 결합하여 결과 반환
    return " ".join(stemming_words)

In [24]:
review_cleaning = review_to_words(train['review'][0])
review_cleaning

'stuff go moment mj start listen music watch odd documentari watch wiz watch moonwalk mayb want get certain insight guy thought realli cool eighti mayb make mind whether guilti innoc moonwalk part biographi part featur film rememb go see cinema origin releas subtl messag mj feel toward press also obvious messag drug bad kay visual impress cours michael jackson unless remot like mj anyway go hate find bore may call mj egotist consent make movi mj fan would say made fan true realli nice actual featur film bit final start minut exclud smooth crimin sequenc joe pesci convinc psychopath power drug lord want mj dead bad beyond mj overheard plan nah joe pesci charact rant want peopl know suppli drug etc dunno mayb hate mj music lot cool thing like mj turn car robot whole speed demon sequenc also director must patienc saint came film kiddi bad sequenc usual director hate work one kid let alon whole bunch perform complex danc scene bottom line movi peopl like mj one level anoth think peopl stay

#### 전체 리뷰에 적용해보자

In [25]:
num_review = train.review.size
num_review

25000

In [28]:
train['review_cleaning'] = train['review'].apply(review_to_words)

오래 걸림..

In [25]:
from multiprocessing import Pool

In [26]:
def _apply_df(args):
    df, func, kwargs = args
    return df.apply(func, **kwargs)

def apply_by_multiprocessing(df, func, **kwargs):
    workers = kwargs.pop('workers')
    pool = Pool(processes=workers)
    result = pool.map(_apply_df, [(d, func, kwargs)
                                 for d in np.array_split(df, workers)])
    pool.close()
    
    return pd.concat(list(result))

In [29]:
test['review_cleaning'] = test['review'].apply(review_to_words)